<a href="https://colab.research.google.com/github/abay-qkt/madb-exploration/blob/main/%E9%80%B1%E5%88%8A%E5%B0%91%E5%B9%B4%E3%82%B8%E3%83%A3%E3%83%B3%E3%83%97%E3%81%AE%E6%8E%B2%E8%BC%89%E9%A0%86%E3%81%AB%E9%96%A2%E3%81%99%E3%82%8B%E8%AA%BF%E6%9F%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

週刊少年ジャンプの作品の掲載順に関する情報を抽出して、pandasやplotlyで集計・可視化してみました。

# 準備

In [1]:
!pip install sparqlwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
from SPARQLWrapper import SPARQLWrapper

# エンドポイントの設定
sparql = SPARQLWrapper(endpoint='https://mediaarts-db.artmuseums.go.jp/sparql', returnFormat='json')

# クエリを投げてDataFrameを返す関数
def read_sparql(query):
  sparql.setQuery(query)
  response = sparql.queryAndConvert()  # 辞書形式で結果を受け取る
  df = pd.DataFrame(response["results"]["bindings"]).applymap(lambda x:x["value"])  # DataFrameに変換する
  df = df[response["head"]["vars"]]  # 列の並び替え
  return df

# 準備

## データロード

In [4]:
query = """
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX class:  <https://mediaarts-db.bunka.go.jp/data/class#>
PREFIX xsd:    <http://www.w3.org/2001/XMLSchema#>

SELECT
    ?公開年月日
    ?ラベル
    ?タイトル
    ?サブタイトル
    ?開始ページ
    ?終了ページ
WHERE {
    ?リソース schema:isPartOf <https://mediaarts-db.bunka.go.jp/id/C119459> ;
    schema:genre "マンガ雑誌単号" ;
    rdfs:label ?ラベル ;
    schema:datePublished ?公開年月日 ;
    schema:hasPart [
        schema:genre "マンガ作品" ;
        schema:name ?タイトル ;
        schema:alternativeHeadline ?サブタイトル ;
        schema:pageStart ?開始ページ ;
        schema:pageEnd ?終了ページ
    ].
}
ORDER BY ?公開年月日 xsd:float(?開始ページ)
"""
df = read_sparql(query)

In [5]:
df.head(10)

,公開年月日,ラベル,タイトル,サブタイトル,開始ページ,終了ページ
0,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,ハレンチ学園,赤い嵐の巻,7.0,37.0
1,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,モサ,盗まれた金の巻,39.0,53.0
2,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,まんがコント55号,ゆうかい魔の巻,54.0,55.0
3,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,どうどう野郎,転校生の巻,56.0,70.0
4,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,男一匹ガキ大将,万吉仁王立ちの巻,71.0,101.0
5,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,赤塚ギャグ笑待席,ゲバゲバ兄弟,103.0,117.0
6,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,鹿兵のケラケラ日記,スポーツの秋 ／ 味覚の秋 ／ ラムダロケットまたしっぱい ／ あみものの季節,118.0,119.0
7,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,デロリンマン,救世主誕生の巻,120.0,134.0
8,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,黄金仮面,くずれる顔,135.0,165.0
9,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,挑戦者ケーン,おれは挑戦する!の巻①,248.0,262.0


In [6]:
df.dtypes

公開年月日     object
ラベル       object
タイトル      object
サブタイトル    object
開始ページ     object
終了ページ     object
dtype: object

いったんすべてobject型で読み込んでいます

## 前処理

In [7]:
# 全部文字列になっているので、適切な型に変換する
df["公開年月日"]=pd.to_datetime(df["公開年月日"])
df["開始ページ"]=df["開始ページ"].astype(float).astype(int)
df["終了ページ"]=df["終了ページ"].astype(float).astype(int)

df["年"]=df["ラベル"].str.extract(r"週刊少年ジャンプ (\d{4})年 表示号数").astype(int)  # 年を抽出
df["号"] = df["ラベル"].str.extract(r"表示号数(\d+)").astype(int)  # 号を抽出
df["年_号"]=df["年"].astype(str)+"_"+df["号"].astype(str).str.zfill(2) # 年_号(2桁)

df=df.sort_values(["年_号","開始ページ"]).reset_index(drop=True)  # ソート

In [8]:
df["作品ページ数"] = df["終了ページ"]-df["開始ページ"] # 各タイトルのページ数
df = df[df["作品ページ数"]>10].reset_index(drop=True)  # 本編以外のものが混ざってそうだったので、ざっくりページ数で判断して除外
df = df.drop_duplicates(subset=["年_号","タイトル"],keep='first').reset_index(drop=True) # 複数同時掲載の場合最初の1本だけ分析対象にする(今回掲載順位を見たいので)

In [9]:
df["掲載順位"] = df.groupby(["年_号"])["開始ページ"].rank().astype(int)  # 掲載順位
df["掲載作品数"] = df.groupby(["年_号"])["タイトル"].transform("count")  # 掲載作品数
df["スコア"] = (df["掲載作品数"]-df["掲載順位"]+1) / df["掲載作品数"]  # 1位が1、最下位が0になるよう正規化した得点を付与

In [10]:
df.head(5)

,公開年月日,ラベル,タイトル,サブタイトル,開始ページ,終了ページ,年,号,年_号,作品ページ数,掲載順位,掲載作品数,スコア
0,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,ハレンチ学園,赤い嵐の巻,7,37,1969,20,1969_20,30,1,9,1.000000
1,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,モサ,盗まれた金の巻,39,53,1969,20,1969_20,14,2,9,0.888889
2,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,どうどう野郎,転校生の巻,56,70,1969,20,1969_20,14,3,9,0.777778
3,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,男一匹ガキ大将,万吉仁王立ちの巻,71,101,1969,20,1969_20,30,4,9,0.666667
4,1969-11-03,週刊少年ジャンプ 1969年 表示号数20,赤塚ギャグ笑待席,ゲバゲバ兄弟,103,117,1969,20,1969_20,14,5,9,0.555556


## 注意事項

In [11]:
# 次の号の公開年月日の方が早いデータが確認された
# ラベルの情報が正しいとすると、公開年月日に誤記があると考えられる
check_cols = ["公開年月日","ラベル","年","号"]
df_magazine = df.drop_duplicates(subset=check_cols)[check_cols].reset_index(drop=True)
df_magazine[df_magazine["公開年月日"].diff().dt.days<0] # 次の号の公開年の方が早いデータの抽出（おそらく誤記）

,公開年月日,ラベル,年,号
1966,2009-10-04,週刊少年ジャンプ 2009年 表示号数45,2009,45
1995,2010-05-17,週刊少年ジャンプ 2010年 表示号数24,2010,24
2042,2011-05-16,週刊少年ジャンプ 2011年 表示号数23,2011,23


In [12]:
# 例えば、2009年は表示号数45から日付がおかしい
df_magazine.loc[1963:1969,:]

,公開年月日,ラベル,年,号
1963,2009-09-28,週刊少年ジャンプ 2009年 表示号数42,2009,42
1964,2009-10-05,週刊少年ジャンプ 2009年 表示号数43,2009,43
1965,2009-10-12,週刊少年ジャンプ 2009年 表示号数44,2009,44
1966,2009-10-04,週刊少年ジャンプ 2009年 表示号数45,2009,45
1967,2009-10-12,週刊少年ジャンプ 2009年 表示号数46,2009,46
1968,2009-10-19,週刊少年ジャンプ 2009年 表示号数47,2009,47
1969,2009-10-26,週刊少年ジャンプ 2009年 表示号数48,2009,48


In [13]:
# 並び順の整合性だけをとったdatetime型のカラムを一応用意しておく
df["疑似年月日"] = pd.to_datetime(df["年"],format='%Y')+pd.to_timedelta(df["号"],unit='W') # 1月1日 + 号＊7

# 可視化

## 掲載作品数の推移

In [14]:
px.line(df.drop_duplicates(["年_号","掲載作品数"]),x='年_号',y='掲載作品数')

## 各年の看板作品

その年に10回より多く掲載されている作品の中で、平均掲載順位が高かった上位5作品を看板作品としてみました。

In [15]:
yearly_rank = (
    df
    .groupby(["年","タイトル"])["掲載順位"]
    .agg(["mean","count"])  # 年ごとに掲載順位の平均値と掲載数を計算
    .rename(columns={"mean":"平均掲載順位","count":"掲載数"})  # カラム名変更
    .query("掲載数>10")  # その年の掲載回数が10より大きい作品のみを抽出
    .reset_index()
    .sort_values(["年","平均掲載順位"],ignore_index=True) # ソート
)
# 平均掲載順位が高い順に順位を振る。同じ値の場合最初に掲載された方を優先して順位を高くする
yearly_rank["年間順位"] = yearly_rank.groupby(["年"])["平均掲載順位"].rank(method='first').astype(int)

# 表示する用に、「タイトル(平均掲載順位)」の形式で文字列を作成
yearly_rank["disp_str"] = yearly_rank.apply(lambda x:"{}({:.1f})".format(x["タイトル"],x["平均掲載順位"]),axis=1)

# 横持ちにして表示
yearly_rank_piv = yearly_rank.pivot(index='年',columns='年間順位',values='disp_str')
yearly_rank_piv.iloc[:,:5]

年間順位,1,2,3,4,5
年,,,,,
1970,ハレンチ学園(2.0),男一匹ガキ大将(4.2),ど根性ガエル(4.4),あらし!三匹(4.7),アニマル球場(5.3)
1971,荒野の少年イサム(1.3),男一匹ガキ大将(2.6),ど根性ガエル(3.5),侍ジャイアンツ(4.1),トイレット博士(4.9)
1972,荒野の少年イサム(2.1),男一匹ガキ大将(2.5),ど根性ガエル(3.1),マジンガーZ(3.4),ハレンチ学園(4.4)
1973,荒野の少年イサム(2.5),大ぼら一代(2.7),マジンガーZ(3.2),ど根性ガエル(3.3),男一匹ガキ大将(3.5)
1974,炎の巨人(2.5),大ぼら一代(3.2),トイレット博士(3.4),プレイボール(3.6),包丁人味平(5.3)
1975,ゼロの白鷹(3.5),ドーベルマン刑事(4.4),トイレット博士(5.0),サーキットの狼(5.1),アストロ球団(5.2)
1976,サーキットの狼(3.3),ドーベルマン刑事(5.1),悪たれ巨人(5.3),四丁目の怪人くん(5.5),1・2のアッホ!!(6.0)
1977,サーキットの狼(2.9),JUMP民話劇場(3.2),こちら葛飾区亀有公園前派出所(4.9),朝太郎伝(5.8),すすめ!!パイレーツ(6.0)
1978,さわやか万太郎(3.6),すすめ!!パイレーツ(4.2),ホールインワン(4.3),サーキットの狼(5.7),ルーズ!ルーズ!!(6.2)


## 各作品の掲載順推移

In [16]:
# 指定したタイトルの順位の時系列プロットを可視化する関数
def plot_rank(df,titles):
  fig = px.line(df[df["タイトル"].isin(titles)],
                x='公開年月日',
                # x='疑似年月日',
                y="掲載順位",color='タイトル',
                hover_data=["タイトル","サブタイトル"])
  fig.update_layout(hovermode="x")
  fig.update_yaxes(autorange='reversed')
  fig.update_traces(
          hovertemplate ="<b>%{y}位</b>：%{customdata[0]}<br>%{customdata[1]}",
          mode='lines+markers',
          marker=dict(line_width=1, size=10))
  fig.update_layout(
      xaxis=dict(
        tickformat="%m月%d日\n%Y",
        rangeslider=dict(visible=True),
        type="date",
        rangeselector=dict(  # ズーム幅をボタンで切り替えられるようにする
          buttons=list([
            dict(count=1,label="1y",step="year",stepmode="backward"),
            dict(count=2,label="2y",step="year",stepmode="backward"),
            dict(count=3,label="3y",step="year",stepmode="backward"),
            dict(step="all")
          ])
        )
      ),
      yaxis=dict(
        fixedrange= True  # y軸のズーム禁止
      )
  )
  fig.show()

In [17]:
# 掲載順位の推移をみるタイトル
target_titles = [
    "ONE PIECE",
    "NARUTO-ナルト-",
    "暗殺教室",
    "僕のヒーローアカデミア"
]

# 可視化
plot_rank(df,target_titles)

## 掲載順位の箱ひげ図

In [18]:
fig = px.box(df[df["タイトル"].isin(target_titles)],x='タイトル',y='掲載順位')
fig.update_yaxes(autorange='reversed')
fig.show()